## SPARQL exercises

Below two exercises that let you explore querying with SPARQL. 

In SPARQL, you have four query types: 
1. SELECT: Return matches as a collection of solution bindings.
2. CONSTRUCT: Create RDF triples from matches.
3. DESCRIBE: Create RDF triples about a resource.
4. ASK: Check if at least one match exists.

## SPARQL exercises

Below two exercises that let you explore querying with SPARQL. 

In SPARQL, you have four query types: 
1. SELECT: Return matches as a collection of solution bindings.
2. CONSTRUCT: Create RDF triples from matches.
3. DESCRIBE: Create RDF triples about a resource.
4. ASK: Check if at least one match exists.

**Exercise 1**

1. Load the recipe.rdf and ingredient.rdf into GraphDB and start the SPARQL service (or you will not be able to query it)
    - tip: load both in a separate named graph, one named graph for ingredients, and one for recipes.  
2. Perform the following queries:
    - SELECT all recipes and their descriptions, ordered alphabetically
    - SELECT all recipes meant for dinner
    - SELECT the recipes that contain the most number of ingredients  
    - ASK if there is any recipe that has a glycemic index of exactly 5. 
    - SELECT recipes that have less than 30 minutes cooking time
    
**Construct queries:** Since a CONSTRUCT query (example below) constructs a graph, and therefor outputs triples instead of a table, you can use it to generate a modified version of the graph, but also to visualise a subgraph. In GraphDB, after running a CONSTRUCT query in the GraphDB SPARQL service, you can select 'Visual' in the lower right corner. 

   - try to visualise an interesting part of the graph: for example a single gluten-free recipe and its ingredients 
In the WHERE part of the query, all subjects ?s and objects ?o will be retrieved that are connected with the property wtm:hasIngredient. The CONSTRUCT part will take ?s and ?o and create a new graph, by again connecting ?s and ?o with wtm:hasIngredient. If you want to create a graph with a _new_ property, you can also do this by replacing the property in the CONSTRUCT part with another one. 


CONSTRUCT { ?s wtm:hasIngredient ?o } 
WHERE { ?s wtm:hasIngredient ?o }



In [44]:
from ast import arg
from rdflib import Graph, RDFS, RDF, URIRef, Namespace, Literal, XSD
from owlrl import DeductiveClosure, RDFS_Semantics
import random, sys
import sys
import os
import pandas as pd
from string import Template
import re
import json

In [60]:
def Query1(graph):
    res = graph.query("""
select ?label where { 
	?s rdf:type wtm:Ingredient .
    ?s rdfs:label ?label 
    } order by asc(?label)
    """, initNs={'wtm': 'http://purl.org/heals/food/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'})

    return list(res)





def Query2(graph):
    res = graph.query("""
select * where { 
    ?s wtm:isRecommendedForMeal wtm:Dinner, wtm:Lunch
} 
    """, initNs={'wtm': 'http://purl.org/heals/food/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'})

    return list(res)


def Query3(graph):
    res = graph.query("""
select ?s  (count(?o) AS ?count)
where { 
    ?s wtm:hasIngredient ?o .
} group by (?s)
having (?count > 9)
    """, initNs={'wtm': 'http://purl.org/heals/food/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'})

    return list(res)



def Query4(graph):
    res = graph.query("""
ASK {
    SELECT ?s (SUM(xsd:double(?g)) as ?sum)  WHERE { 
        ?s wtm:hasIngredient ?i . 
        ?i  wtm:hasGlycemicIndex ?g .  
	} group by ?s
	having (?sum > xsd:double("300")) 
}

    """, initNs={'wtm': 'http://purl.org/heals/food/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#', 'xsd': 'http://www.w3.org/2001/XMLSchema#'})

    return list(res)



def Query5(graph):
    res = graph.query("""
select * where { 
	?s wtm:hasCookTime ?o .
    FILTER(?o < 30)
}
    """, initNs={'wtm': 'http://purl.org/heals/food/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'})

    return list(res)



def Query6(graph):
    res = graph.query("""
CONSTRUCT 
	{ ?r wtm:hasIngredient ?i } 
WHERE {
    
    ?r wtm:hasIngredient ?i . 
    
    {
        SELECT ?r {
        ?r wtm:hasIngredient ?i . 
        MINUS { ?r wtm:hasIngredient/wtm:hasGluten True }   
        } LIMIT 1
    } 
} 

    """, initNs={'wtm': 'http://purl.org/heals/food/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'})

    return list(res)



In [62]:
g = Graph()
g.parse("../data/ingredients.rdf")
g.parse("../data/recipes.rdf")




Query5(g)

[(rdflib.term.URIRef('http://purl.org/heals/ingredient/ChickenSalad'),
  rdflib.term.Literal('15', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))),
 (rdflib.term.URIRef('http://purl.org/heals/ingredient/ThaiChicken'),
  rdflib.term.Literal('15', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))),
 (rdflib.term.URIRef('http://purl.org/heals/ingredient/WholeGrainBananaPancake'),
  rdflib.term.Literal('20', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))]

**Exercise 2**
1. Explore the [LMDB](https://triplydb.com/Triply/linkedmdb) dataset
2. Perform the following queries:
    - How many classes are there? 
    - Which are the distinguishing properties of a <https://triplydb.com/Triply/linkedmdb/vocab/Film> class? 
    - Can you find the 5 actors that performed in most movies, and the name of these movies ? 
    - What is the IRI of “Tom Hanks (Actor)” ?
    - Can you find the IRI of movies and with their DBpedia correspondent IRI?

In [64]:
def Query(graph):
    res = graph.query("""
SELECT
  DISTINCT ?s ?p ?o
WHERE
  { 
    SERVICE <http://linkeddata.uriburner.com/sparql> 
      {
        SELECT ?s ?p ?o
        WHERE {?s ?o ?o}
        LIMIT 100                
      }
  }
    """, initNs={'wtm': 'http://purl.org/heals/food/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'})

    return list(res)


d = Graph()
d.parse('http://linkeddata.uriburner.com/sparql')  

Query(d)

[(rdflib.term.URIRef('http://www.openlinksw.com/schemas/graphql/intro#__Type'),
  None,
  rdflib.term.URIRef('http://www.openlinksw.com/schemas/graphql/intro#fields')),
 (rdflib.term.URIRef('urn:records:test#'),
  None,
  rdflib.term.URIRef('urn:records:test#')),
 (rdflib.term.URIRef('https://sklep.sizeer.com/air-jordan-1-mid-meskie-sneakersy-bialy-554724-092'),
  None,
  rdflib.term.URIRef('https://sklep.sizeer.com/nowosci/kategoria-akcesoria')),
 (rdflib.term.URIRef('https://sklep.sizeer.com/air-jordan-1-mid-meskie-sneakersy-bialy-554724-092'),
  None,
  rdflib.term.URIRef('https://sklep.sizeer.com/nowosci/kategoria-buty')),
 (rdflib.term.URIRef('https://sklep.sizeer.com/air-jordan-1-mid-meskie-sneakersy-bialy-554724-092'),
  None,
  rdflib.term.URIRef('https://sklep.sizeer.com/nowosci/kategoria-odziez')),
 (rdflib.term.URIRef('http://www.wikidata.org/prop/direct/P1647'),
  None,
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subPropertyOf')),
 (rdflib.term.URIRef('http:/

In [73]:
def Query(graph):
    res = graph.query("""
SELECT
  DISTINCT ?s ?p ?o
WHERE
  { 
    SERVICE <https://api.triplydb.com/datasets/Triply/linkedmdb/services/linkedmdb/sparql> 
      {
        SELECT *
        WHERE {?s ?o ?o}
        LIMIT 10               
      }
  }
    """, initNs={'wtm': 'http://purl.org/heals/food/', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'})

    return list(res)


d = Graph()
d.parse('https://api.triplydb.com/datasets/Triply/linkedmdb')  

Query(d)

HTTPError: HTTP Error 400: Expected 'query' argument.